In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix


In [27]:
class SpotifyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SpotifyRankPredictor(nn.Module):
    def __init__(self, num_categories):
        super(SpotifyRankPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16, num_categories)  # 출력층이 1개에서 5개(카테고리 수)로 변경
        )
        self.softmax = nn.Softmax(dim=1)  # Softmax 활성화 함수 추가하여 각 카테고리의 확률 출력

    def forward(self, x):
        x = self.layers(x)
        return self.softmax(x)

In [28]:
def augment_features(features, category, num_augmentations):
    augmented_data = []
    feature_names = ['Danceability', 'Energy', 'Loudness', 'Speechiness',
                     'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']

    noise_ranges = {
        'Danceability': 0.05,
        'Energy': 0.05,
        'Loudness': 1.0,
        'Speechiness': 0.02,
        'Acousticness': 0.05,
        'Liveness': 0.05,
        'Tempo': 3.0,
        'Duration (ms)': 0.05  # 5% 변화
    }
    # features의 각 행에 대해 증강을 수행
    for feature_row in features:
        for _ in range(num_augmentations):
            new_features = []
            for feat_idx, feat_name in enumerate(feature_names):
                feature = np.abs(feature_row[feat_idx])  # 개별 행의 특성값 사용

                # 나머지 로직은 동일...
                if feat_name == 'Loudness':
                    noise = np.random.normal(0, np.abs(noise_ranges['Loudness']))
                elif feat_name == 'Tempo':
                    noise = np.random.normal(0, np.abs(noise_ranges['Tempo']))
                elif feat_name == 'Duration (ms)':
                    noise = np.random.normal(0, np.abs(feature * noise_ranges['Duration (ms)']))
                else:
                    noise = np.random.normal(0, np.abs(noise_ranges[feat_name]))

                new_value = feature + noise

                if feat_name == 'Loudness':
                    new_value = np.clip(new_value, -60, 0)
                elif feat_name == 'Duration (ms)':
                    new_value = max(1000, new_value)
                else:
                    new_value = np.clip(new_value, 0, 1)

                new_features.append(new_value)

            augmented_data.append(new_features)

    return np.array(augmented_data)

In [29]:
def preprocess_data_with_augmentation(df):

    X = df[['Danceability', 'Energy', 'Loudness', 'Speechiness',
            'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']].values

    def rank_to_category(rank):
        if rank <= 10:
            return 0
        elif rank <= 30:
            return 1
        elif rank <= 50:
            return 2
        elif rank <= 100:
            return 3
        else:
            return 4

    ranks = df['Highest Charting Position'].values
    categories = np.array([rank_to_category(rank) for rank in ranks])

    # 각 카테고리별 증강 비율 설정
    augmentation_ratios = {
        0: 5,  # Top 10
        1: 4,  # Top 11-30
        2: 3,  # Top 31-50
        3: 2,  # Top 51-100
        4: 0   # Below 100
    }


    augmented_features = []
    augmented_categories = []

    for category in range(5):
        category_mask = categories == category
        category_features = X[category_mask]

        if category != 4:  # Below 100이 아닌 경우에만 증강
            # 증강 데이터 생성
            new_features = augment_features(
                category_features,
                category,
                len(category_features) * augmentation_ratios[category]
            )

            augmented_features.append(new_features)
            augmented_categories.extend([category] * len(new_features))

    # 원본 데이터와 증강 데이터 합치기
    if augmented_features:
        augmented_features = np.vstack(augmented_features)
        X_combined = np.vstack([X, augmented_features])
        categories_combined = np.concatenate([categories, augmented_categories])
    else:
        X_combined = X
        categories_combined = categories

    # 특성 스케일링
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_combined)

    # 원-핫 인코딩
    num_categories = 5
    y_encoded = np.eye(num_categories)[categories_combined]

    return X_scaled, y_encoded, num_categories

In [30]:
def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=100, patience=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)  # Classification 문제로 바뀌었기 때문에 이 부분 수정. 기존에는 y_batch(-1,1)여서 연속적인 값 예측시 사용하는 것
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                _, actual = torch.max(y_batch.data, 1)
                total += y_batch.size(0)
                correct += (predicted == actual).sum().item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        accuracy = 100 * correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.2f}%')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping triggered')
                break

    return train_losses, val_losses

In [31]:
def main():
    df = pd.read_csv('spotify_dataset.csv')
    X_scaled, y_encoded, num_categories = preprocess_data_with_augmentation(df)

    X_train, X_temp, y_train, y_temp = train_test_split(
        X_scaled, y_encoded, test_size=0.2, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42
    )

    train_dataset = SpotifyDataset(X_train, y_train)
    val_dataset = SpotifyDataset(X_val, y_val)
    test_dataset = SpotifyDataset(X_test, y_test)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    model = SpotifyRankPredictor(num_categories)
    criterion = nn.CrossEntropyLoss()  # 분류 문제여서 기존 MSE 에서 CrossEntropy로 바꿈
    optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)

    train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=100,
        patience=10
    )

    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            total += y_batch.size(0)
            correct += (predicted == actual).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

    y_pred = []
    y_true = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            y_pred.extend(predicted.numpy())
            y_true.extend(actual.numpy())

    conf_matrix = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

if __name__ == "__main__":
    main()

Epoch [1/100], Train Loss: 1.4426, Val Loss: 1.4151, Val Accuracy: 47.88%
Epoch [2/100], Train Loss: 1.4272, Val Loss: 1.4068, Val Accuracy: 48.56%
Epoch [3/100], Train Loss: 1.4238, Val Loss: 1.3998, Val Accuracy: 49.37%
Epoch [4/100], Train Loss: 1.4192, Val Loss: 1.3967, Val Accuracy: 49.86%
Epoch [5/100], Train Loss: 1.4182, Val Loss: 1.3981, Val Accuracy: 49.69%
Epoch [6/100], Train Loss: 1.4191, Val Loss: 1.3941, Val Accuracy: 50.20%
Epoch [7/100], Train Loss: 1.4178, Val Loss: 1.3927, Val Accuracy: 50.14%
Epoch [8/100], Train Loss: 1.4172, Val Loss: 1.3953, Val Accuracy: 49.81%
Epoch [9/100], Train Loss: 1.4161, Val Loss: 1.3929, Val Accuracy: 50.11%
Epoch [10/100], Train Loss: 1.4159, Val Loss: 1.3946, Val Accuracy: 50.00%
Epoch [11/100], Train Loss: 1.4161, Val Loss: 1.3944, Val Accuracy: 50.02%
Epoch [12/100], Train Loss: 1.4152, Val Loss: 1.3903, Val Accuracy: 50.36%
Epoch [13/100], Train Loss: 1.4149, Val Loss: 1.3921, Val Accuracy: 50.29%
Epoch [14/100], Train Loss: 1.4149

C:\Users\이강민\AppData\Local\Temp\ipykernel_6480\3380637125.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Accuracy: 52.60%

Confusion Matrix:
[[10157  5714     9 15919     0]
 [ 3501 18867    23 19787     0]
 [ 1986  3784   147 10693     0]
 [ 4379  7528    20 52299     0]
 [    2     9     0    62     0]]
